# Distributed Data Parallel

DistributedDataParallel (DDP) implementa el paralelismo de datos a nivel de módulo, lo que permite ejecutarlo en múltiples máquinas. Las aplicaciones que utilizan DDP deben crear múltiples procesos y crear una única instancia de DDP por proceso. DDP utiliza comunicaciones colectivas en el paquete ```torch.distributed``` para sincronizar gradientes y buffers. Específicamente, DDP registra un autograd hook para cada parámetro dado por ```model.parameters()```, y el hook se activa cuando se calcula el gradiente correspondiente en el paso de retropropagación. Luego, DDP utiliza esa señal para desencadenar la sincronización de gradientes entre los procesos. 

La forma recomendada de utilizar DDP es crear un proceso para cada réplica del modelo, donde una réplica del modelo puede abarcar múltiples dispositivos. Los procesos de DDP se pueden ubicar en la misma máquina o en máquinas diferentes, pero las GPU no se pueden compartir entre procesos. Este tutorial parte de un caso de uso básico de DDP y luego demuestra casos de uso más avanzados, como el guardado de modelos y la combinación de DDP con la división de modelos.

In [1]:
import sys
import tempfile
import torch
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

from torch.nn.parallel import DistributedDataParallel as DDP

In [2]:
from defs import *

In [3]:
def run_demo(demo_fn, world_size):
    mp.spawn(demo_fn,
             args=(world_size,),
             nprocs=world_size,
             join=True)

In [4]:
if __name__ == "__main__":
    n_gpus = torch.cuda.device_count()
    assert n_gpus >= 2, f"Requires at least 2 GPUs to run, but got {n_gpus}"
    world_size = n_gpus
    run_demo(demo_basic, world_size)

Running basic DDP example on rank 1.
Running basic DDP example on rank 0.
